In [4]:
import requests
import time

In [13]:
def get_request(url, parameters=None):
    print("Your request is being processed.")
    response = requests.get(url=url, params=parameters)
    if response:
        return response.json()
    else:
        print("Request unsuccessful, retrying in 5 seconds.")
        time.sleep(5)
        return get_request(url, parameters)
    time.sleep(5)


In [19]:
url = "http://store.steampowered.com/api/appdetails/"
parameters = {"appids": "406210"}
data = get_request(url, parameters=parameters)
data

Your request is being processed.


{'406210': {'success': True,
  'data': {'type': 'video',
   'name': 'Over the Dream',
   'steam_appid': 406210,
   'required_age': 0,
   'is_free': False,
   'controller_support': 'full',
   'detailed_description': '<h1>Steam Video</h1><p>This content is only available in an online streaming format. More information about streaming videos can be found in the <a href="https://support.steampowered.com/kb_article.php?ref=8699-OASD-1871">Streaming Videos on Steam FAQ</a>.</p><br><h1>About the Video</h1>The International, a legendary tournament, which had already become an annual celebration for all Dota 2 fans and spectators, came back in July of 2015. This time four teams from CIS were honored to prove themselves in a battle for 18 million dollars. But our story is about only one team – Virtus.pro. However, The International is not only about teams. Anton &quot;Mob5ter&quot; Erohin, who didn’t even know what Dota 2 was several years ago, had already become one of very few Russian &quot;vo